<a href="https://colab.research.google.com/github/Moukthika1253/Facial_Expression_ImageClassifier/blob/main/facial_expression_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import seaborn as sns
import tensorflow as tf

In [3]:
train_data="/content/images/train"
test_data="/content/images/validation"
train_count=[]
exp_list=["angry","surprise","neutral","sad","happy","fear","disgust"]
print("Images in train set:")
for emotion in os.listdir(train_data):
  train_count.append(len(os.listdir(train_data+"/"+emotion)))

sns.barplot(x=exp_list,y=train_count)

Images in train set:


FileNotFoundError: ignored

In [12]:
test_count=[]
for emotion in os.listdir(test_data):
  test_count.append(len(os.listdir(test_data+"/"+emotion)))
sns.barplot(x=exp_list,y=test_count)

Images in test set:
angry : 960
surprise : 797
neutral : 1216
sad : 1139
happy : 1825
fear : 1018
disgust : 111


**Image Generator**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
img_size=48
batchSize=